In [38]:
import requests
import json

Set the drug that we want to find associations for

In [39]:
myDrug = "INFLIXIMAB"

Make the API call to find all the drugs

In [40]:
size = 1000
link = 'https://www.targetvalidation.org/api/latest/public/evidence/filter?datatype=known_drug&size='
url = link + str(size)

response = requests.get(url)
output = response.json()

#print out the json for debug
#print json.dumps(output, indent=2)

Collect information about diseases and targets associated with our drug

In [42]:
total = output['total']
fromCount = size - 1
diseases = {}
targets = {}

while (fromCount < total):
    
    if('data' in output):
        for target in output['data']:
        
            if(myDrug.lower() == target['drug']['molecule_name'].lower()):
                for x in target['disease']['efo_info']['therapeutic_area']['labels']:
                    diseases[x] = 'true'
                
                targets[target['target']['gene_info']['symbol']] = 'true'
                #print "Drug: " + target['drug']['molecule_name']
                #print ', '.join([str(x) for x in target['disease']['efo_info']['therapeutic_area']['labels']])
                #print "Target: " + target['target']['gene_info']['symbol']
    
    fromCount = fromCount + size
    url = link + str(size) + "&from=" + str(fromCount)

    response = requests.get(url)
    output = response.json()
    #print json.dumps(output, indent=2)
    
print "DRUG: " + myDrug

print "\nDISEASES: "

for key in diseases.keys():
    print "\t" + key
    
print "\nTARGETS:"

for key in targets.keys():
    print "\t" + key

DRUG: INFLIXIMAB

DISEASES: 
	eye disease
	infectious disease
	skeletal system disease
	reproductive system disease
	immune system disease
	respiratory system disease
	skin disease
	cardiovascular disease
	metabolic disease
	digestive system disease

TARGETS:
	TNF
